In [6]:
from dash_website import MAIN_CATEGORIES_TO_CATEGORIES
from dash_website.utils.aws_loader import load_feather, upload_file

all_categories = MAIN_CATEGORIES_TO_CATEGORIES["All"] + [f"All_{main_category}" for main_category in MAIN_CATEGORIES_TO_CATEGORIES.keys()]

for category in all_categories:
    correlations = load_feather(f"xwas/univariate_correlations/correlations/categories/correlations_{category}.feather").drop(
        columns=[
            "r2_1",
            "r2_std_1",
            "r2_2",
            "r2_std_2",
        ]
    )

    for dimension, subdimension in [("Lungs", "Spirometry"), ("Hearing", "HearingTest"), ("BloodCells", "BloodCount")]:
        for i in [1, 2]:
            correlations.loc[correlations[f"dimension_{i}"] == dimension, f"subdimension_{i}"] = subdimension
        

    scores = load_feather(f"age_prediction_performances/scores_scores_all_samples_per_participant.feather").set_index(
        ["dimension", "subdimension", "sub_subdimension", "algorithm"]
    )
    scores.drop(index=scores.index[~scores.index.isin(CUSTOM_DIMENSIONS)], inplace=True)
    scores.drop(index=scores.index[scores.index.get_level_values("algorithm") != "*"], inplace=True)
    scores.reset_index(["sub_subdimension", "algorithm"], drop=True, inplace=True)

    for number in [1, 2]:
        correlations.set_index([f"dimension_{number}", f"subdimension_{number}"], inplace=True)
        correlations[f"r2_{number}"] = scores["r2"]
        correlations[f"r2_std_{number}"] = scores["r2_std"]
        correlations.reset_index(inplace=True)

    return correlations.to_dict()
    break

           dimension_2 subdimension_2       dimension_1 subdimension_1  \
0                    *              *                 *              *   
1         *instances01              *                 *              *   
2       *instances1.5x              *                 *              *   
3         *instances23              *                 *              *   
4              Abdomen              *                 *              *   
...                ...            ...               ...            ...   
1084   Musculoskeletal           Hips  PhysicalActivity              *   
1085   Musculoskeletal          Knees  PhysicalActivity              *   
1086   Musculoskeletal        Scalars  PhysicalActivity              *   
1087   Musculoskeletal          Spine  PhysicalActivity              *   
1088  PhysicalActivity              *  PhysicalActivity              *   

      ('all', 'pearson')  ('all', 'spearman')  ('all', 'number_variables')  \
0               1.000000         